In [49]:
import langgraph
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph,END
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage,AnyMessage,ToolMessage
from langgraph.checkpoint.sqlite import SqliteSaver
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated,List
import operator
from IPython import Image,display


In [59]:
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPEN_API_KEY2")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [51]:
tools=TavilySearchResults(max_results=4)

In [52]:
tools.name

'tavily_search_results_json'

In [53]:
class AgentState(TypedDict):
    messages:Annotated[List[AnyMessage],operator.add]


In [54]:
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        print("we are in exists_action")
        print(state['messages'][-1])
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
                print(f"{t['name']}-{t['args']}")
                print(result)
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [61]:
model = ChatOpenAI(model="gpt-3.5-turbo")
prompt="""You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
abot = Agent(model=model,tools=[tools],system=prompt)

In [67]:
messages = [HumanMessage(content="in which year pakistan won the t20 world cup? and in which is held and what was the weather at that time")]
# print("Input to invoke:", {"messages": AgentState.message})

result = abot.graph.invoke({"messages": messages})

we are in exists_action
content='' additional_kwargs={'tool_calls': [{'id': 'call_Ec14Vap8hNq0GutvTRoBOH8G', 'function': {'arguments': '{"query": "Pakistan T20 World Cup win year"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_QB8SbPyhOhHzcAglkn5tLMUj', 'function': {'arguments': '{"query": "T20 World Cup location Pakistan win"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 170, 'total_tokens': 233, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-14c9b2cf-3c52-4d26-a64b-ccbf873759a9-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Pakistan T20 World Cup win year'}, 'id': 'call_Ec14Vap8hNq0GutvTRoBOH8G', 'type':

In [68]:
result["messages"][-1].content

"Pakistan won the T20 World Cup in the year 2009. The tournament was held in England. Unfortunately, I couldn't find specific information about the weather during that time."